In [1]:
using DataFrames, LinearAlgebra, Distributions, Revise, Serialization, DebuggingUtilities

In [2]:
const datadir = "/export/storage_adgandhi/MiscLi/factract";
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

---

## Computation

In [3]:
Threads.nthreads()

32

In [4]:
testmodes = ["full", "fl", "fl17"]
testmode = testmodes[3]################# EDIT THIS ##################

pars = deserialize("jls/pars.jls");
ec = deserialize("$datadir/analysis/ec$testmode.jls");

In [5]:
σ = ones(2);

In [51]:
pars = deserialize("jls/pars.jls");
ec = deserialize("$datadir/analysis/ec$testmode.jls");

deltas, q_iter = m.compute_deltas(ec, pars, σ, max_iter=10000, verbose=true);

# serialize("jls/deltas.jls", deltas);

dist = 9.989390719056246e-7, iterations = 5516


In [46]:
# verifying log sum exp trick
utils = [2,10,8]
expu = exp.(utils)
@showln maximum(expu)
denom = 1 .+ sum(expu, dims=1)
share_i = expu ./ denom
shares = mean(share_i, dims=2)


maximum(expu) = 22026.465794806718
(in /users/facsupport/zhli/blp-quantities/mean_utils.ipynb:3)


3-element Vector{Float64}:
 0.0002953754150143755
 0.880501702562868
 0.11916294730666543

In [49]:
utils = [2,10,8]
util_reduction = max(median(utils), 0.)
utils .-= util_reduction
expu = exp.(utils)
@showln maximum(expu)
scale = exp(-util_reduction)
denom = sum(expu) .+ scale 
share_i = expu ./ denom
shares = mean(share_i, dims=2)


maximum(expu) = 7.38905609893065
(in /users/facsupport/zhli/blp-quantities/mean_utils.ipynb:5)


3-element Vector{Float64}:
 0.00029537541501437563
 0.8805017025628682
 0.11916294730666546

In [13]:
scale = exp.(-util_reduction)

0.1353352832366127

In [ ]:
using BenchmarkTools

In [ ]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

In [ ]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 10
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

In [ ]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1000
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

In [ ]:
Threads.nthreads()

In [ ]:
# # this was the line that the memory profiler said was creating the vast majority of allocations
# # ...which doesn't make sense because it's not even in the inner loop
# # ...and it's the inner loop that's driving the allocations and times (according to benchmarks)
# @time for t in ec.tracts
#     t.abδ .= t.D * pars.nlcoefs
# end

___
continued in `gmm.ipynb`